In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PrevisaoVendasHackathon").getOrCreate()

df_transacoes = spark.read.parquet("dados/transacoes.parquet")
df_produtos = spark.read.parquet("dados/cadastroprodutos.parquet")
df_pdvs = spark.read.parquet("dados/cadastropdv.parquet")

df_transacoes.printSchema()
df_transacoes.show(5)
df_produtos.printSchema()
df_produtos.show(5)
df_pdvs.printSchema()
df_pdvs.show(5) 

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/18 22:34:21 WARN Utils: Your hostname, DESKTOP-GV5311B, resolves to a loopback address: 127.0.1.1; using 172.19.89.69 instead (on interface eth0)
25/09/18 22:34:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/18 22:34:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- internal_store_id: string (nullable = true)
 |-- internal_product_id: string (nullable = true)
 |-- distributor_id: string (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- reference_date: date (nullable = true)
 |-- quantity: double (nullable = true)
 |-- gross_value: double (nullable = true)
 |-- net_value: double (nullable = true)
 |-- gross_profit: double (nullable = true)
 |-- discount: double (nullable = true)
 |-- taxes: double (nullable = true)



+-------------------+-------------------+--------------+----------------+--------------+--------+------------------+------------------+------------------+------------------+------------------+
|  internal_store_id|internal_product_id|distributor_id|transaction_date|reference_date|quantity|       gross_value|         net_value|      gross_profit|          discount|             taxes|
+-------------------+-------------------+--------------+----------------+--------------+--------+------------------+------------------+------------------+------------------+------------------+
|7384367747233276219| 328903483604537190|             9|      2022-07-13|    2022-07-01|     1.0|            38.125|         37.890625|10.042625427246094| 3.950000047683716|          0.234375|
|3536908514005606262|5418855670645487653|             5|      2022-03-21|    2022-03-01|     6.0|            107.25|106.44000244140625| 24.73200225830078|17.100000381469727|0.8100000023841858|
|3138231730993449825|10870055626757

#### Tratamento de Colunas com Nomes Duplicados: `categoria`

**Problema Identificado:**
Foi observado que tanto o DataFrame `df_produtos` quanto o `df_pdvs` possuem uma coluna com o mesmo nome: `categoria`.

**Risco Potencial:**
Manter nomes de colunas duplicados pode causar problemas de ambiguidade ao unir (fazer `join`) esses DataFrames. Se unirmos as tabelas, o DataFrame resultante terá duas colunas `categoria`, tornando difícil e propenso a erros referenciar a coluna correta em análises e transformações futuras.


In [2]:
df_produtos_renomeado = df_produtos.withColumnRenamed("categoria", "categoria_produto")
df_pdvs_renomeado = df_pdvs.withColumnRenamed("categoria", "categoria_pdv")

df_produtos_renomeado.show(5)
df_pdvs_renomeado.show(5)

+-------------------+-----------------+--------------------+-----------------+-------------+-------------------+--------------------+--------------------+
|            produto|categoria_produto|           descricao|            tipos|        label|       subcategoria|               marca|          fabricante|
+-------------------+-----------------+--------------------+-----------------+-------------+-------------------+--------------------+--------------------+
|2282334733936076502|Distilled Spirits|JOSEPH CARTRON CA...|Distilled Spirits|         Core|Liqueurs & Cordials| Joseph Cartron Cafe|            Spiribam|
|6091840953834683482|Distilled Spirits|SPRINGBANK 18 YEA...|Distilled Spirits|    Specialty|      Scotch Whisky|Springbank 18 Yea...|Pacific Edge Wine...|
|1968645851245092408|Distilled Spirits|J BRANDT TRIPLE S...|Distilled Spirits|Private Label|Liqueurs & Cordials| J Brandt Triple Sec|     Sazerac Spirits|
| 994706710729219179|            Draft|REFORMATION CASHM...|          

#### Unificação dos Dados: Junção das Tabelas de Transações, Produtos e PDVs

Nesta etapa, consolidamos as informações de transações, produtos e pontos de venda (PDVs) em um único DataFrame chamado `df_completo`. Para isso, realizamos duas operações de junção (`join`):

1.  **Junção com Dados dos Produtos:**
    *   **Operação:** Unimos `df_transacoes` com `df_produtos_renomeado`.
    *   **Chave:** `internal_product_id` (de transações) e `produto` (de produtos).
    *   **Tipo:** `left join`. Isso garante que todas as transações sejam mantidas, mesmo que um produto não seja encontrado no cadastro. Nesses casos, as colunas de produto (`categoria_produto`, `descricao`, etc.) ficarão com valor `null`.

2.  **Junção com Dados dos PDVs:**
    *   **Operação:** O resultado da primeira junção é então unido com `df_pdvs_renomeado`.
    *   **Chave:** `internal_store_id` (de transações) e `pdv` (de PDVs).
    *   **Tipo:** `left join`. Da mesma forma, mantemos todas as transações, preenchendo com `null` as informações de PDVs não cadastrados.

O resultado é um DataFrame enriquecido, onde cada linha de transação agora contém detalhes completos sobre o produto vendido e o ponto de venda onde a venda ocorreu.

In [3]:
df_completo = df_transacoes.join(
    df_produtos_renomeado,
    df_transacoes["internal_product_id"] == df_produtos_renomeado["produto"], 
    "left"
).join(
    df_pdvs_renomeado,
    df_transacoes["internal_store_id"] == df_pdvs_renomeado["pdv"],
    "left"
)

In [4]:
print("Schema do DataFrame unificado:")
df_completo.printSchema()

print("\nAmostra dos dados unificados:")
df_completo.show(5)

Schema do DataFrame unificado:
root
 |-- internal_store_id: string (nullable = true)
 |-- internal_product_id: string (nullable = true)
 |-- distributor_id: string (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- reference_date: date (nullable = true)
 |-- quantity: double (nullable = true)
 |-- gross_value: double (nullable = true)
 |-- net_value: double (nullable = true)
 |-- gross_profit: double (nullable = true)
 |-- discount: double (nullable = true)
 |-- taxes: double (nullable = true)
 |-- produto: string (nullable = true)
 |-- categoria_produto: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- tipos: string (nullable = true)
 |-- label: string (nullable = true)
 |-- subcategoria: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- fabricante: string (nullable = true)
 |-- pdv: string (nullable = true)
 |-- premise: string (nullable = true)
 |-- categoria_pdv: string (nullable = true)
 |-- zipcode: integer (nullable = true

#### Limpeza Pós-Junção: Remoção de Colunas de ID Redundantes

**Contexto:**
Após a unificação dos DataFrames, o `df_completo` resultante contém as colunas que foram utilizadas como chaves para a junção. Especificamente:
-   A coluna `produto` (vinda de `df_produtos_renomeado`) é idêntica à `internal_product_id`.
-   A coluna `pdv` (vinda de `df_pdvs_renomeado`) é idêntica à `internal_store_id`.

**Ação Proposta:**
Para manter o conjunto de dados limpo e evitar redundância, as colunas `produto` e `pdv` serão removidas. Essa prática simplifica o schema do DataFrame, tornando-o mais fácil de ser compreendido e manipulado nas etapas seguintes de análise e modelagem.

In [5]:
df_completo = df_completo.drop("produto").drop("pdv")

In [6]:
print("Schema do DataFrame unificado:")
df_completo.printSchema()

print("\nAmostra dos dados unificados:")
df_completo.show(5)

Schema do DataFrame unificado:
root
 |-- internal_store_id: string (nullable = true)
 |-- internal_product_id: string (nullable = true)
 |-- distributor_id: string (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- reference_date: date (nullable = true)
 |-- quantity: double (nullable = true)
 |-- gross_value: double (nullable = true)
 |-- net_value: double (nullable = true)
 |-- gross_profit: double (nullable = true)
 |-- discount: double (nullable = true)
 |-- taxes: double (nullable = true)
 |-- categoria_produto: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- tipos: string (nullable = true)
 |-- label: string (nullable = true)
 |-- subcategoria: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- fabricante: string (nullable = true)
 |-- premise: string (nullable = true)
 |-- categoria_pdv: string (nullable = true)
 |-- zipcode: integer (nullable = true)


Amostra dos dados unificados:
+-------------------+-------------------

In [7]:
#verificacoes basicas de quantidade e integridade
from pyspark.sql.functions import col, count, when, isnan, isnull

#tamanho do dataset
total_transacoes = df_completo.count()
print(f"Número total de transações em 2022: {total_transacoes}")

#verificar se os joins criaram valores nulos(o que indicaria transações com produtos ou PDVs não cadastrados)
# a quantidade que der null (join falhou), significa que as transacoes são vendas de lojas "fantasmas" ou não cadastradas
# venda sem ponto de venda
print("\nContagem de valores nulos após o join:")
df_completo.select([
    count(when(isnull(c), c)).alias(c) for c in ["categoria_produto", "categoria_pdv", "zipcode"]
]).show()

Número total de transações em 2022: 6560698

Contagem de valores nulos após o join:


+-----------------+-------------+-------+
|categoria_produto|categoria_pdv|zipcode|
+-----------------+-------------+-------+
|                0|        45582|  45582|
+-----------------+-------------+-------+



#### Tratamento de Transações com PDVs Não Cadastrados

**Análise do Problema:**
A contagem de valores nulos na célula anterior revelou que **45.582 transações** não possuem informações correspondentes de Ponto de Venda (PDV), como `categoria_pdv` e `zipcode`. Isso significa que o `left join` entre as transações e o cadastro de PDVs não encontrou uma correspondência para essas vendas.

**Impacto no Conjunto de Dados:**
Essas 45.582 linhas representam aproximadamente **0.7%** do total de transações (6.560.698). Trata-se de uma fração muito pequena do volume total de dados.

**Opções de Tratamento:**

1.  **Remoção das Linhas:**
    *   **Prós:** É uma abordagem simples e segura. Garante que o modelo de previsão treinará apenas com dados completos e consistentes, onde cada venda está associada a um PDV conhecido.
    *   **Contras:** Perda de uma pequena quantidade de dados de transação.

2.  **Imputação de Dados:**
    *   **Prós:** Mantém o número total de transações.
    *   **Contras:** Seria difícil e arriscado atribuir um PDV a essas vendas. Uma imputação inadequada (ex: usar um valor como "Desconhecido") poderia introduzir ruído e vieses no modelo, prejudicando sua precisão.

**Decisão:**
Dado que o percentual de dados afetados é mínimo e o risco de introduzir informações incorretas com a imputação é alto, a abordagem mais prudente é **remover as linhas** com informações de PDV ausentes. Essa ação garantirá a integridade do dataset para as próximas etapas de análise e modelagem, com um impacto insignificante na quantidade total de dados.

In [8]:
df_completo_limpo = df_completo.na.drop(subset=["categoria_pdv"])

print(f"Tamanho original: {df_completo.count()}")
print(f"Tamanho após limpeza: {df_completo_limpo.count()}")

Tamanho original: 6560698


Tamanho após limpeza: 6515116


In [9]:
df_completo_limpo.printSchema()
df_completo_limpo.show(5)

root
 |-- internal_store_id: string (nullable = true)
 |-- internal_product_id: string (nullable = true)
 |-- distributor_id: string (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- reference_date: date (nullable = true)
 |-- quantity: double (nullable = true)
 |-- gross_value: double (nullable = true)
 |-- net_value: double (nullable = true)
 |-- gross_profit: double (nullable = true)
 |-- discount: double (nullable = true)
 |-- taxes: double (nullable = true)
 |-- categoria_produto: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- tipos: string (nullable = true)
 |-- label: string (nullable = true)
 |-- subcategoria: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- fabricante: string (nullable = true)
 |-- premise: string (nullable = true)
 |-- categoria_pdv: string (nullable = true)
 |-- zipcode: integer (nullable = true)



+-------------------+-------------------+--------------+----------------+--------------+--------+------------------+------------------+------------------+------------------+------------------+-----------------+--------------------+-----------------+-----+-------------------+--------------------+--------------------+-----------+--------------+-------+
|  internal_store_id|internal_product_id|distributor_id|transaction_date|reference_date|quantity|       gross_value|         net_value|      gross_profit|          discount|             taxes|categoria_produto|           descricao|            tipos|label|       subcategoria|               marca|          fabricante|    premise| categoria_pdv|zipcode|
+-------------------+-------------------+--------------+----------------+--------------+--------+------------------+------------------+------------------+------------------+------------------+-----------------+--------------------+-----------------+-----+-------------------+-------------------

#### Foco da Análise Exploratória: Dimensões-Chave para a Previsão

O objetivo central deste projeto é prever a **quantidade de vendas semanal** para cada **produto (SKU)** em cada **Ponto de Venda (PDV)**.

Portanto, nossa Análise Exploratória de Dados (EDA) será direcionada para encontrar padrões e insights relacionados a três dimensões fundamentais:

-   **O Quê? (`quantity`)**: A quantidade de produtos vendidos em cada transação. Analisaremos sua distribuição, outliers e comportamento geral.
-   **Onde? (`internal_store_id`)**: O ponto de venda onde a transação ocorreu. Investigaremos a concentração de vendas por loja e possíveis padrões geográficos ou de categoria de PDV.
-   **Quando? (`transaction_date`)**: A data da transação. Exploraremos sazonalidades, tendências ao longo do tempo e padrões semanais, que são cruciais para a modelagem de séries temporais.

A análise focada nessas três dimensões nos permitirá extrair as características (features) mais relevantes para construir um modelo de previsão preciso.

#### Análise da Variável Alvo: `quantity`

Nossa primeira investigação foca na variável `quantity`, que representa a quantidade de itens vendidos por transação. O objetivo é entender sua distribuição estatística e identificar possíveis anomalias.

**Questões a serem respondidas:**
-   Qual é a distribuição das quantidades vendidas?
-   Existem valores atípicos (outliers) que possam distorcer a análise ou o modelo?


In [10]:

df_completo_limpo.select("quantity").describe().show()
# quantity que significa a quantidade vendida por CADA transacao 

+-------+-----------------+
|summary|         quantity|
+-------+-----------------+
|  count|          6515116|
|   mean|8.149641143197064|
| stddev|80.71476955086146|
|    min|          -1530.0|
|    max|          94230.0|
+-------+-----------------+



#### Análise das Estatísticas Descritivas de `quantity`


**Observações Principais:**

*   **Alta Variabilidade e Outliers:**
    *   A **média** de itens por transação é de aproximadamente **8**.
    *   O **desvio padrão** é superior a **80**, mais de 10 vezes o valor da média. Essa disparidade indica uma enorme variabilidade nos dados e a presença de outliers extremos.
    *   O **valor máximo** registrado é de quase **100.000** itens em uma única transação. Isso sugere que o dataset pode conter não apenas vendas a consumidores finais, mas também transações de maior volume, como vendas B2B ou movimentações entre centros de distribuição.

*   **Presença de Devoluções:**
    *   O **valor mínimo** é negativo, indicando a existência de transações de devolução ou estorno.

**Decisões e Próximos Passos:**

Para garantir que o modelo de previsão seja treinado com dados representativos de vendas reais, é necessário tratar as anomalias identificadas:

1.  **Remover Devoluções:** As transações com `quantity` negativa ou nula não representam vendas e devem ser removidas. Elas introduzem ruído e não contribuem para o objetivo de prever vendas futuras. A célula seguinte executará essa filtragem.


In [11]:

print(f"Tamanho do DataFrame antes de filtrar as quantidades: {df_completo_limpo.count()}")

# Filtra o DataFrame para manter apenas as linhas onde a quantidade é maior que zero
df_vendas_reais = df_completo_limpo.filter(df_completo_limpo["quantity"] > 0)

print(f"Tamanho do DataFrame após filtrar (apenas vendas reais): {df_vendas_reais.count()}")

quantidade_removida = df_completo_limpo.count() - df_vendas_reais.count()
print(f"Número de transações removidas (quantidade <= 0): {quantidade_removida}")

# Agora, vamos rodar as estatísticas descritivas novamente neste novo DataFrame limpo
print("\nEstatísticas descritivas apenas para vendas reais:")
df_vendas_reais.select("quantity").describe().show()

Tamanho do DataFrame antes de filtrar as quantidades: 6515116


Tamanho do DataFrame após filtrar (apenas vendas reais): 6423880


Número de transações removidas (quantidade <= 0): 91236

Estatísticas descritivas apenas para vendas reais:


+-------+--------------------+
|summary|            quantity|
+-------+--------------------+
|  count|             6423880|
|   mean|   8.298247866619342|
| stddev|   81.26202216095105|
|    min|1.192092895507812...|
|    max|             94230.0|
+-------+--------------------+



#### Investigação de Quantidades Fracionadas

Após a remoção das devoluções, a análise estatística da coluna `quantity` ainda apresenta uma peculiaridade: o valor mínimo não é um número inteiro. Isso é inesperado, pois a quantidade de produtos vendidos geralmente é contada em unidades inteiras.

**Hipótese:**
Existem duas possibilidades principais para a ocorrência de valores fracionados:

1.  **Vendas por Peso/Volume:** O conjunto de dados pode incluir produtos que são vendidos por peso ou volume (ex: frutas, frios, produtos a granel), onde quantidades como `0.5` kg são perfeitamente válidas.
2.  **Ruído no Tipo de Dado:** Os valores podem ser resultado de imprecisões de ponto flutuante (tipo `double`), onde um valor que deveria ser `5` é armazenado como `5.00000001`.


In [12]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType


# filtragem do df para encontrar linhas onde a coluna 'quantity' não é um número inteiro
# (quantity!= int(quantity)) faz exatamente essa verificação.
df_quantidades_fracionadas = df_vendas_reais.filter(
    col("quantity")!= col("quantity").cast(IntegerType())
)

contagem_fracionadas = df_quantidades_fracionadas.count()

if contagem_fracionadas > 0:
    print(f"{contagem_fracionadas} transações com quantidades que não são inteiras.")
    print("amostra:")
    
    df_quantidades_fracionadas.select("quantity", "categoria_produto", "descricao").show(truncate=False)
else:
    print("nenhuma transação com quantidade fracionada foi encontrada.")

95346 transações com quantidades que não são inteiras.
amostra:
+------------------+-----------------+--------------------------------------------------------------+
|quantity          |categoria_produto|descricao                                                     |
+------------------+-----------------+--------------------------------------------------------------+
|45.9999897480011  |Distilled Spirits|BLANTON'S BOURBON 6/750ML 93PF                                |
|14.000010132789612|Distilled Spirits|CARAVELLA LIMONCELLO 6/750ML 56PF                             |
|139.99979883432388|Distilled Spirits|FIREBALL 12/1L 66PF                                           |
|79.00000804662704 |Distilled Spirits|FIREBALL 12/1L 66PF                                           |
|5.9999880194664   |Distilled Spirits|GRAN GALA ORANGE LIQUEUR12/L 80PF                             |
|83.0000039935112  |Distilled Spirits|PEYCHAUD'S APERTIVO 12/750ML 22PF                             |
|5.999999880790710

In [13]:
from pyspark.sql.functions import rand

print("Exibindo 20 linhas aleatórias do DataFrame:")

# 1. orderBy(rand()) embaralha o DataFrame de forma aleatória.
# 2. show(20) pega as 20 primeiras linhas do DataFrame já embaralhado.
df_quantidades_fracionadas.orderBy(rand()).select("quantity", "categoria_produto", "descricao").show(20)

Exibindo 20 linhas aleatórias do DataFrame:
+------------------+-----------------+--------------------+
|          quantity|categoria_produto|           descricao|
+------------------+-----------------+--------------------+
| 7.999997973442078|Distilled Spirits|THOMAS S MOORE CH...|
| 1.899999976158142|          Package|BUSCH LIGHT 30/12 CN|
|               0.5|          Package|  BUDWEISER 12/32 NR|
|2.0000040531158447|Distilled Spirits|BITTERMENS HELLFI...|
|15.000000059604645|Distilled Spirits|PAUL MASSON APPLE...|
| 70.99999797344208|Distilled Spirits|PAUL MASSON BRAND...|
| 35.99999403953552|Distilled Spirits|BLANTON'S BOURBON...|
| 6.000012159347534|Distilled Spirits|BENCHMARK NO 8 BO...|
|   0.9999960064888|Distilled Spirits|HIGHLAND MIST SCO...|
|26.000004053115845|Distilled Spirits|SEAGRAM'S VO CANA...|
| 981.9999959468842|Distilled Spirits|PAUL MASSON BRAND...|
|236.89583300054073|Distilled Spirits|PLATINUM 7X VODKA...|
|   0.9999960064888|Distilled Spirits|SOUTHERN COMFORT .

### Conclusão da Análise e Decisão de Tratamento

**Análise da Amostra Aleatória:**
A amostra aleatória exibida na célula anterior confirma que as quantidades fracionadas não estão restritas a categorias específicas de produtos, como itens vendidos por peso. Pelo contrário, elas aparecem em uma vasta gama de produtos que são vendidos em unidades inteiras.

**Confirmação da Hipótese:**
Isso fortalece a hipótese de que os valores fracionados são, na verdade, **ruído de ponto flutuante** (imprecisões do tipo `double`), e não representam vendas fracionadas legítimas. Por exemplo, um valor que deveria ser `5` é armazenado como `5.00000001`.

**Ação Corretiva:**
Para garantir a consistência e a precisão dos dados para a modelagem, a abordagem correta é converter a coluna `quantity` para o tipo `Integer`. Essa ação irá truncar a parte decimal irrelevante, limpando o ruído e garantindo que as quantidades de vendas sejam representadas como valores inteiros.

A célula seguinte executará essa conversão.

In [14]:
from pyspark.sql.types import IntegerType
# .withColumn() substitui a coluna existente pela nova versão convertida.
df_vendas_final = df_vendas_reais.withColumn("quantity", df_vendas_reais["quantity"].cast(IntegerType()))

# Agora, vamos verificar o schema e as estatísticas novamente para confirmar a mudança.
print("\nSchema do DataFrame final (após cast para inteiro):")
df_vendas_final.printSchema()

print("\nEstatísticas descritivas do DataFrame final:")
df_vendas_final.select("quantity").describe().show()


Schema do DataFrame final (após cast para inteiro):
root
 |-- internal_store_id: string (nullable = true)
 |-- internal_product_id: string (nullable = true)
 |-- distributor_id: string (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- reference_date: date (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- gross_value: double (nullable = true)
 |-- net_value: double (nullable = true)
 |-- gross_profit: double (nullable = true)
 |-- discount: double (nullable = true)
 |-- taxes: double (nullable = true)
 |-- categoria_produto: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- tipos: string (nullable = true)
 |-- label: string (nullable = true)
 |-- subcategoria: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- fabricante: string (nullable = true)
 |-- premise: string (nullable = true)
 |-- categoria_pdv: string (nullable = true)
 |-- zipcode: integer (nullable = true)


Estatísticas descritivas do DataFrame final:


+-------+-----------------+
|summary|         quantity|
+-------+-----------------+
|  count|          6423880|
|   mean|8.290080294152444|
| stddev| 81.2568283392018|
|    min|                0|
|    max|            94230|
+-------+-----------------+



In [15]:
df_vendas_final_limpo = df_vendas_final.filter(col("quantity") > 0)

In [16]:
print("Mostrando transações com quantidade menor que 2 (ou seja, quantidade = 1):")

# Filtra o DataFrame para manter apenas as linhas onde a coluna 'quantity' é menor que 2
df_vendas_pequenas = df_vendas_final_limpo.filter(df_vendas_final_limpo["quantity"] < 2)

# Mostra as primeiras 20 linhas do resultado
df_vendas_pequenas.show()

# Se quiser saber quantas transações são de apenas 1 item:
print(f"Número de transações com apenas 1 item: {df_vendas_pequenas.count()}")



Mostrando transações com quantidade menor que 2 (ou seja, quantidade = 1):


+-------------------+-------------------+--------------+----------------+--------------+--------+------------------+------------------+------------------+-------------------+-------------------+-----------------+--------------------+-----------------+------------+---------------+--------------------+--------------------+-----------+--------------+-------+
|  internal_store_id|internal_product_id|distributor_id|transaction_date|reference_date|quantity|       gross_value|         net_value|      gross_profit|           discount|              taxes|categoria_produto|           descricao|            tipos|       label|   subcategoria|               marca|          fabricante|    premise| categoria_pdv|zipcode|
+-------------------+-------------------+--------------+----------------+--------------+--------+------------------+------------------+------------------+-------------------+-------------------+-----------------+--------------------+-----------------+------------+---------------+----

Número de transações com apenas 1 item: 2888148


### Investigação e Remoção do Outlier Extremo (`quantity` = 94.230)

**Contexto:**
A análise estatística final no DataFrame `df_vendas_final_limpo` (realizada na célula 26) revelou um valor máximo (`max`) de **94.230** para a coluna `quantity`. Este valor é um outlier extremo e distorce completamente a escala da variável, considerando que:
-   A **média** de itens por transação é de aproximadamente **8**.
-   **75%** de todas as transações envolvem **6 itens ou menos**.

Uma única transação de mais de 94 mil unidades não representa um comportamento de compra de varejo típico. É muito provável que seja um registro de outra natureza, como uma venda B2B (atacado), uma transferência de estoque entre um centro de distribuição e uma loja, ou simplesmente um erro de entrada de dados.

**Ação Proposta:**
Para continuar a análise focada no comportamento do consumidor final, é prudente remover essa transação anômala. Essa remoção nos permitirá reavaliar as estatísticas descritivas e ter uma visão mais clara da distribuição das vendas de varejo, que constituem a esmagadora maioria dos nossos dados.

A célula seguinte irá filtrar o DataFrame para excluir essa única linha e, em seguida, recalcular as estatísticas para que possamos entender o impacto dessa limpeza.

In [17]:
df_sem_anomalia = df_vendas_final_limpo.filter(col("quantity")!= 94230)
print(f"Tamanho do DataFrame após remover a anomalia: {df_sem_anomalia.count()}")
df_sem_anomalia.select("quantity").describe().show()

Tamanho do DataFrame após remover a anomalia: 6407822


+-------+-----------------+
|summary|         quantity|
+-------+-----------------+
|  count|          6407822|
|   mean|8.296149768205172|
| stddev|72.34358197762367|
|    min|                1|
|    max|            32424|
+-------+-----------------+



In [18]:
# Usando o DataFrame que não tem a anomalia de 94230
# df_sem_anomalia = df_vendas_final_limpo.filter(col("quantity")!= 94230)

print("Analisando os percentis da coluna 'quantity' com mais detalhes (sem a anomalia):")

# Pedimos pontos bem no topo da distribuição: 98.5%, 99.5%, 99.9%
percentis_finais = df_sem_anomalia.approxQuantile(
    "quantity", [0.985, 0.995, 0.999], 0.01
)

print(f"Percentil 98.5%: {percentis_finais[0]}")
print(f"Percentil 99.5%: {percentis_finais[1]}")
print(f"Percentil 99.9%: {percentis_finais[2]}")

Analisando os percentis da coluna 'quantity' com mais detalhes (sem a anomalia):


Percentil 98.5%: 89.0
Percentil 99.5%: 32424.0
Percentil 99.9%: 32424.0


In [19]:
from pyspark.sql.functions import col

# Começamos com o DataFrame que já não tem a primeira anomalia
# df_sem_anomalia = df_vendas_final_limpo.filter(col("quantity")!= 94230)

print("Removendo a transação com quantidade = 32424...")

# Filtra o DataFrame para remover a linha onde a quantidade é 32424
df_varejo_real = df_sem_anomalia.filter(col("quantity")!= 32424)

print(f"Número de transações no DataFrame original (sem 94230): {df_sem_anomalia.count()}")
print(f"Número de transações após remover o 32424: {df_varejo_real.count()}")


# Agora, vamos rodar as estatísticas descritivas novamente neste DataFrame mais limpo
print("\nEstatísticas descritivas após remover o segundo outlier:")
df_varejo_real.select("quantity").describe().show()

Removendo a transação com quantidade = 32424...


Número de transações no DataFrame original (sem 94230): 6407822


Número de transações após remover o 32424: 6407821

Estatísticas descritivas após remover o segundo outlier:


+-------+-----------------+
|summary|         quantity|
+-------+-----------------+
|  count|          6407821|
|   mean|8.291090996455738|
| stddev|71.20120063626807|
|    min|                1|
|    max|            32070|
+-------+-----------------+



#### Estratégia de Tratamento de Outliers: Da Remoção Manual ao Capping

A remoção manual de outliers, um a um, como foi feito nas células anteriores, se mostrou um processo reativo e pouco eficiente. A cada remoção, um novo valor máximo extremo surgia, indicando que o dataset possui uma "cauda longa" de transações de altíssimo volume que não representam o comportamento de compra do consumidor final.

Para tratar esse problema de forma sistemática e robusta, adotaremos uma técnica chamada **capping** (ou "limitação de teto").


#### Por que usar Capping em vez de Remover?

-   **Preserva a Informação:** Em vez de descartar a transação inteira, o capping mantém o registro, mas neutraliza o efeito distorcivo do valor extremo. Ele preserva o sinal de que aquela foi uma "venda de grande volume".
-   **Protege o Modelo:** Impede que valores raros e extremos desequilibrem as métricas estatísticas (como média e desvio padrão) e, mais importante, evita que o modelo de machine learning seja enviesado por esses pontos anômalos.

A análise de percentis realizada anteriormente nos fornecerá um limite data-driven para definir um teto de quantidade razoável, separando as vendas de varejo (mesmo as de alto volume) de eventos logísticos ou de atacado.

In [20]:
from pyspark.sql.functions import col

# Definir o limite de quantidade que queremos verificar
limite = 89.0

# Filtrar o DataFrame para encontrar transações com quantidade > 250 e contá-las
contagem = df_sem_anomalia.filter(col("quantity") > limite).count()

print(f"Número de transações com quantidade acima de {limite}: {contagem}")

Número de transações com quantidade acima de 89.0: 84587


In [21]:
from pyspark.sql.functions import when, col

# df_varejo_real é o DataFrame que você acabou de criar, já sem os dois maiores outliers

# Definimos nosso teto com base na análise de percentis do varejo real
teto_quantidade = 89.0
print(f"\nAnálise finalizada. Aplicando um teto de {teto_quantidade} na coluna 'quantity'...")

# Aplicamos o capping para tratar todos os outliers de alto volume restantes
df_tratado = df_varejo_real.withColumn(
    "quantity_tratada",
    when(col("quantity") > teto_quantidade, teto_quantidade)
 .otherwise(col("quantity"))
)

# Verificamos as estatísticas finais para confirmar
print("\nEstatísticas descritivas finais após a remoção dos outliers e capping:")
df_tratado.select("quantity_tratada").describe().show()


Análise finalizada. Aplicando um teto de 89.0 na coluna 'quantity'...

Estatísticas descritivas finais após a remoção dos outliers e capping:


+-------+------------------+
|summary|  quantity_tratada|
+-------+------------------+
|  count|           6407821|
|   mean| 5.399676114548144|
| stddev|12.572682686033117|
|    min|               1.0|
|    max|              89.0|
+-------+------------------+



In [22]:
#TODO mudar tamanho dos titulos e verificar no vscode

Análise das Dimensões Principais: PDV e Produto
Agora, vamos entender os atores principais: os pontos de venda e os produtos.

pergunta: quais são os produtos e PDVs mais vendidos? A venda está concentrada em poucos itens/lojas?

Por que é útil? É provável que a "Lei de Pareto" (80/20) se aplique: uma minoria de produtos/PDVs é responsável pela maioria das vendas. Identificá-los é crucial. Talvez os produtos "campeões de venda" tenham um padrão de venda diferente dos produtos de "cauda longa".

In [23]:
# Contar o número de PDVs (lojas) únicos
num_pdvs = df_tratado.select("internal_store_id").distinct().count()
print(f"Número total de PDVs únicos: {num_pdvs}")

# Contar o número de produtos (SKUs) únicos
num_produtos = df_tratado.select("internal_product_id").distinct().count()
print(f"Número total de produtos únicos: {num_produtos}")

Número total de PDVs únicos: 14388


Número total de produtos únicos: 6927


Insights sobre os Produtos (A Tabela "Top 10 Produtos")
Domínio Absoluto de Cervejas: A lista é completamente dominada por algumas marcas de cerveja, principalmente BUD LIGHT, MICHELOB ULTRA e BUSCH LIGHT. Isso nos diz que o negócio principal gira em torno da venda de cervejas populares de grande volume.

Concentração de Vendas (Princípio de Pareto): O produto número 1, BUD LIGHT 24/12 CN, vendeu quase 1 milhão de unidades, quase o dobro do segundo colocado. Isso é um exemplo clássico do Princípio de Pareto (ou regra 80/20): um número muito pequeno de produtos é responsável por uma fatia gigantesca do total de vendas.

Importância das Embalagens: As descrições (24/12 CN, 30/12 CN, 15/16 ALM NR) indicam que os produtos mais vendidos são pacotes grandes (multi-packs). Isso sugere que o consumidor típico está comprando para estocar, e não para consumo imediato.

Conclusão sobre os Produtos: Seu modelo de previsão precisa ser muito bom em prever a demanda para esses poucos produtos "campeões". Um erro na previsão da Bud Light tem um impacto muito maior no negócio do que um erro em um produto de nicho.

Insights sobre os PDVs (A Tabela "Top 10 PDVs")
Canal de Venda Principal: A descoberta mais importante aqui é que todos os 10 PDVs mais importantes pertencem à mesma categoria: Package/Liquor. Isso significa que o principal canal de vendas são as lojas de bebidas, onde os clientes compram para levar para casa (off-premise).

Distribuição Mais Equilibrada: Diferente dos produtos, a venda nos PDVs é mais distribuída. A loja número 1 vendeu 66 mil unidades, enquanto a décima vendeu 52 mil. Não há uma única loja que domine as outras de forma tão esmagadora quanto a Bud Light domina os produtos.

Consistência do Canal: A consistência na categoria dos PDVs mais importantes é uma pista valiosa. O comportamento de compra nessas lojas tende a ser semelhante, o que pode facilitar a modelagem.

Conclusão sobre os PDVs: A feature categoria_pdv será extremamente importante para o seu modelo. Ele precisa aprender que lojas do tipo Package/Liquor têm um padrão de venda muito diferente de outros tipos de loja que possam existir no dataset (como lojas de conveniência, etc.

In [24]:
from pyspark.sql.functions import sum, col

# Top 10 produtos por quantidade total vendida
print("\n--- Top 10 Produtos por Quantidade Total Vendida ---")
df_tratado.groupBy("internal_product_id", "descricao") \
 .agg(sum("quantity_tratada").alias("quantidade_total")) \
 .orderBy(col("quantidade_total").desc()) \
 .show(10, truncate=False)

# Top 10 PDVs por quantidade total vendida
print("\n--- Top 10 PDVs por Quantidade Total Vendida ---")
df_tratado.groupBy("internal_store_id", "categoria_pdv") \
 .agg(sum("quantity_tratada").alias("quantidade_total")) \
 .orderBy(col("quantidade_total").desc()) \
 .show(10, truncate=False)


--- Top 10 Produtos por Quantidade Total Vendida ---


+-------------------+----------------------------+----------------+
|internal_product_id|descricao                   |quantidade_total|
+-------------------+----------------------------+----------------+
|4040509988492387426|BUD LIGHT 24/12 CN          |983383.0        |
|8352471677482341950|MICHELOB ULTRA 24/12 CN     |567709.0        |
|3894706280449257667|BUSCH LIGHT 30/12 CN        |544663.0        |
|500478784353013717 |BUD LIGHT 2/15/12 CN        |506546.0        |
|3262679882836704514|BUD LIGHT 2/12/12 LN        |467895.0        |
|1029370090212151375|MICHELOB ULTRA 2/12/12 CN   |457065.0        |
|1860061817666925715|BUDWEISER 24/12 CN          |441555.0        |
|8899583048247637290|BUD LIGHT 15/16 ALM NR      |418435.0        |
|1657665165780983454|BUD LIGHT 24/12 LNNR DISPLAY|416108.0        |
|1152772499033912340|BUD LIGHT 2/12/12 CN        |370200.0        |
+-------------------+----------------------------+----------------+
only showing top 10 rows

--- Top 10 PDVs por Qu

+-------------------+--------------+----------------+
|internal_store_id  |categoria_pdv |quantidade_total|
+-------------------+--------------+----------------+
|4374038751643985193|Package/Liquor|66214.0         |
|7195906766187577140|Package/Liquor|65634.0         |
|3025867614395044464|Package/Liquor|65608.0         |
|6491855528940268514|Package/Liquor|65335.0         |
|6337402841339348330|Package/Liquor|63786.0         |
|8723723113467008071|Package/Liquor|59879.0         |
|5130630496972372280|Package/Liquor|56905.0         |
|8294871217390043140|Package/Liquor|55093.0         |
|5420232849866774947|Package/Liquor|53393.0         |
|338592148842312906 |Package/Liquor|52009.0         |
+-------------------+--------------+----------------+
only showing top 10 rows


In [25]:
print("Salvando o DataFrame tratado em um arquivo Parquet...")

df_tratado.write.mode("overwrite").parquet("dados_processados/dados_tratados.parquet")

print("DataFrame salvo com sucesso!")

Salvando o DataFrame tratado em um arquivo Parquet...


DataFrame salvo com sucesso!
